In [2]:
import socket
from zeroconf import ServiceBrowser, ServiceStateChange, Zeroconf


class PiDiscovery:
    def __init__(self):
        self.pis = {}
        self.zeroconf = Zeroconf()
        self.browser = ServiceBrowser(
            self.zeroconf,
            "_pivideo._tcp.local.",
            handlers=[self.on_service_state_change],
        )
        
    def on_service_state_change(self, zeroconf, service_type, name, state_change):
        if state_change is ServiceStateChange.Added:
            info = self.zeroconf.get_service_info(service_type, name)
            if info:
                ip = socket.inet_ntoa(info.addresses[0])
                hostname = info.properties.get(b"hostname", b"").decode("utf-8")
                self.pis[hostname] = f"http://{ip}:8000"

    def get_pis(self):
        return self.pis


discovery = PiDiscovery()


In [3]:
discovery.get_pis().items()

dict_items([])

In [ ]:
from zeroconf import Zeroconf, ServiceBrowser, ServiceStateChange
import socket


class PiDiscovery:
    def __init__(self):
        self.pis = {}
        self.zeroconf = Zeroconf()
        self.browser = ServiceBrowser(
            self.zeroconf,
            "_pivideo._tcp.local.",
            handlers=[self.on_service_state_change],
        )

    def on_service_state_change(self, zeroconf, service_type, name, state_change):
        if state_change is ServiceStateChange.Added:
            info = zeroconf.get_service_info(service_type, name)
            if info:
                ip = socket.inet_ntoa(info.addresses[0])
                hostname = info.properties.get(b"hostname", b"").decode("utf-8")
                self.pis[hostname] = ip
        elif state_change is ServiceStateChange.Removed:
            self.pis.pop(name, None)

    def get_pis(self):
        return [{"name": name, "host": host} for name, host in self.pis.items()]


if __name__ == "__main__":
    import time

    discovery = PiDiscovery()

    try:
        print("Scanning for Pis...")
        while True:
            time.sleep(5)
            print("Discovered Pis:", discovery.get_pis())
    except KeyboardInterrupt:
        print("Stopping discovery.")
    finally:
        discovery.zeroconf.close()


Scanning for Pis...
Discovered Pis: []
Stopping discovery.


Exception in thread zeroconf-ServiceBrowser-_pivideo._tcp-142730:
Exception in thread zeroconf-ServiceBrowser-_pivideo._tcp-143298:
Traceback (most recent call last):
  File "/home/r1j1n/miniconda3/envs/dtv/lib/python3.13/threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "src/zeroconf/_services/browser.py", line 811, in zeroconf._services.browser.ServiceBrowser.run
  File "src/zeroconf/_services/browser.py", line 730, in zeroconf._services.browser._ServiceBrowserBase._fire_service_state_changed_event
  File "src/zeroconf/_services/browser.py", line 740, in zeroconf._services.browser._ServiceBrowserBase._fire_service_state_changed_event
  File "src/zeroconf/_services/__init__.py", line 56, in zeroconf._services.Signal.fire
  File "/tmp/ipykernel_142697/1764594277.py", line 17, in on_service_state_change
    info = zeroconf.get_service_info(service_type, name)
  File "/home/r1j1n/miniconda3/envs/dtv/lib/python3.13/site-packages/zeroconf/_core.py", line 27

In [7]:
import os
time.ctime(os.path.getmtime("videos/ah shit here we go again.mp4"))

'Mon Jan 13 13:26:47 2025'